In [1]:
# import the necessary packages
import imutils
import time
import cv2

In [2]:
# load the neural style transfer model from disk
net = cv2.dnn.readNetFromTorch("models/eccv16/starry_night.t7")
# load the input image, resize it to have a width of 600 pixels, and
# then grab the image dimensions
image = cv2.imread("images/lena.jpg")
image = imutils.resize(image, width=600)
(h, w) = image.shape[:2]
# construct a blob from the image, set the input, and then perform a
# forward pass of the network
blob = cv2.dnn.blobFromImage(image, 1.0, (w, h),
	(103.939, 116.779, 123.680), swapRB=False, crop=False)
net.setInput(blob)
start = time.time()
output = net.forward()
end = time.time()

In [3]:
# reshape the output tensor, add back in the mean subtraction, and
# then swap the channel ordering
output = output.reshape((3, output.shape[2], output.shape[3]))
output[0] += 103.939
output[1] += 116.779
output[2] += 123.680
output /= 255.0
output = output.transpose(1, 2, 0)

In [5]:
# show information on how long inference took
print("[INFO] neural style transfer took {:.4f} seconds".format(
	end - start))
# show the images
cv2.imshow("Input", image)
cv2.imshow("Output", output)
cv2.waitKey()
cv2.destroyAllWindows()

[INFO] neural style transfer took 0.4529 seconds


In [1]:
# import the necessary packages
from imutils.video import VideoStream
from imutils import paths
import itertools
import imutils
import time
import cv2

In [2]:
# grab the paths to all neural style transfer models in our 'models'
# directory, provided all models end with the '.t7' file extension
modelPaths = paths.list_files("models", validExts=(".t7",))
modelPaths = sorted(list(modelPaths))
# generate unique IDs for each of the model paths, then combine the
# two lists together
models = list(zip(range(0, len(modelPaths)), (modelPaths)))
# use the cycle function of itertools that can loop over all model
# paths, and then when the end is reached, restart again
modelIter = itertools.cycle(models)
(modelID, modelPath) = next(modelIter)

net = cv2.dnn.readNetFromTorch(modelPath)


In [3]:
# # load the neural style transfer model from disk
# print("[INFO] loading style transfer model...")
# net = cv2.dnn.readNetFromTorch(modelPath)
# # initialize the video stream, then allow the camera sensor to warm up
# print("[INFO] starting video stream...")
# time.sleep(2.0)
# print("[INFO] {}. {}".format(modelID + 1, modelPath))

In [4]:
vs = VideoStream(src=0).start()

# loop over frames from the video file stream
while True:
	# grab the frame from the threaded video stream
	frame = vs.read()
	# resize the frame to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	frame = imutils.resize(frame, width=600)
	orig = frame.copy()
	(h, w) = frame.shape[:2]
	# construct a blob from the frame, set the input, and then perform a
	# forward pass of the network
	blob = cv2.dnn.blobFromImage(frame, 1.0, (w, h),
		(103.939, 116.779, 123.680), swapRB=False, crop=False)
	net.setInput(blob)
	output = net.forward()
	# reshape the output tensor, add back in the mean subtraction, and
	# then swap the channel ordering
	output = output.reshape((3, output.shape[2], output.shape[3]))
	output[0] += 103.939
	output[1] += 116.779
	output[2] += 123.680
	output /= 255.0
	output = output.transpose(1, 2, 0)
	# show the original frame along with the output neural style
	# transfer
	cv2.imshow("Input", frame)
	cv2.imshow("Output", output)
	key = cv2.waitKey(1) & 0xFF
	# if the `n` key is pressed (for "next"), load the next neural
	# style transfer model
	if key == ord("n"):
		# grab the next neural style transfer model model and load it
		(modelID, modelPath) = next(modelIter)
		print("[INFO] {}. {}".format(modelID + 1, modelPath))
		net = cv2.dnn.readNetFromTorch(modelPath)
	# otheriwse, if the `q` key was pressed, break from the loop
	elif key == ord("q"):
		break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()